In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import csv


: 

In [ ]:
# change these 3 to generate flights to each city for the date from the specific airport
departloc = "MSN"
depart_time = "2025-01-4"
return_time = "2025-01-6"

def generate_flight_links(departloc, depart_time, return_time, team_airports):
    flight_links = {}
    
    for team, arriveloc in team_airports.items():
        flight_url = f"https://flights.booking.com/flights/{departloc}.AIRPORT-{arriveloc}.AIRPORT/?type=ROUNDTRIP&adults=1&cabinClass=ECONOMY&children=&from={departloc}.AIRPORT&to={arriveloc}.AIRPORT&fromCountry=US&toCountry=US&fromLocationName=Dane+County+Regional+Airport&toLocationName=Green+Bay%E2%80%93Austin+Straubel+International+Airport&depart={depart_time}&return={return_time}&sort=BEST&travelPurpose=leisure&aid=304142&label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4AvnvprsGwAIB0gIkMGI3ZTY2OTItMjdlYS00M2JjLThiZTAtOTRjODRhMDA5NzVh2AIE4AIB&adplat=www-searchresults_irene-web_shell_header-flight-missing_creative-5VHWvGC1SMhs4k08O29emO&stops=1"
        flight_links[team] = flight_url

    return flight_links

team_airports = {
    "MIN": "MSP",  # Minnesota Vikings - Minneapolis-St. Paul International
    "CHI": "ORD",  # Chicago Bears - O'Hare International
    "GB": "GRB",   # Green Bay Packers - Green Bay–Austin Straubel International
    "DET": "DTW",  # Detroit Lions - Detroit Metropolitan
    "IND": "IND",  # Indianapolis Colts - Indianapolis International
    "JAX": "JAX",  # Jacksonville Jaguars - Jacksonville International
    "TEN": "BNA",  # Tennessee Titans - Nashville International
    "NE": "BOS",   # New England Patriots - Logan International
    "BUF": "BUF",  # Buffalo Bills - Buffalo Niagara International
    "MIA": "MIA",  # Miami Dolphins - Miami International
    "NYJ": "JFK",  # New York Jets - John F. Kennedy International
    "BAL": "BWI",  # Baltimore Ravens - Baltimore/Washington International
    "CIN": "CVG",  # Cincinnati Bengals - Cincinnati/NKY International
    "CLE": "CLE",  # Cleveland Browns - Cleveland Hopkins International
    "PIT": "PIT",  # Pittsburgh Steelers - Pittsburgh International
    "HOU": "IAH",  # Houston Texans - George Bush Intercontinental
    "KC": "MCI",   # Kansas City Chiefs - Kansas City International
    "LV": "LAS",   # Las Vegas Raiders - McCarran International
    "LAC": "LAX",  # Los Angeles Chargers - Los Angeles International
    "LAR": "LAX",  # Los Angeles Rams - Los Angeles International
    "SEA": "SEA",  # Seattle Seahawks - Seattle-Tacoma International
    "ARI": "PHX",  # Arizona Cardinals - Phoenix Sky Harbor International
    "SF": "SFO",   # San Francisco 49ers - San Francisco International
    "ATL": "ATL",  # Atlanta Falcons - Hartsfield-Jackson Atlanta International
    "CAR": "CLT",  # Carolina Panthers - Charlotte Douglas International
    "NO": "MSY",   # New Orleans Saints - Louis Armstrong New Orleans International
    "TB": "TPA",   # Tampa Bay Buccaneers - Tampa International
    "NYG": "JFK",  # New York Giants - John F. Kennedy International
    "DAL": "DFW",  # Dallas Cowboys - Dallas/Fort Worth International
    "WAS": "DCA",  # Washington Commanders - Ronald Reagan Washington National
    "PHI": "PHL",  # Philadelphia Eagles - Philadelphia International
    "CHI": "ORD",  # Chicago Bears - O'Hare International
    "MIN": "MSP",  # Minnesota Vikings - Minneapolis-St. Paul International
}

flight_links = generate_flight_links(departloc, depart_time, return_time, team_airports)

for team, flight_url in flight_links.items():
    print(f"Flight link for {team}: {flight_url}")
